<a href="https://colab.research.google.com/github/Haniyesabeghi/Maral-LLM-Project/blob/main/Maral_7B_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Maral 7B Inference Notebook

<p align="center">
 <img src="https://huggingface.co/MaralGPT/Maral-7B-alpha-1/resolve/main/maral-7b-announce.png" width=256 height=256/>
</p>

## About Maral

Maral is just a new large lanugage model, specializing on the Persian language. This model is based on Mistral and trained an Alpaca Persian dataset. This model is one of the few efforts in Persian speaking scene in order to bring our language to a new life in the era of AI.

Also, since Maral is based on Mistral, it's capable of producing English answers as well.

## Our Team

* Muhammadreza Haghiri ([Website](https://haghiri75.com/en) - [Github](https://github.com/prp-e) - [LinkedIn](https://www.linkedin.com/in/muhammadreza-haghiri-1761325b))
* Mahi Mohrechi ([Website](https://mohrechi-portfolio.vercel.app/) - [Github](https://github.com/f-mohrechi) - [LinkedIn](https://www.linkedin.com/in/faeze-mohrechi/))

## Needed libraries

Since the model is loaded in 8 bit quantization mode on free colab, you need `bitsandbytes`. If you do own a better GPU, go with full 16 bit quantization.

In [1]:
!pip install transformers accelerate bitsandbytes -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 20.8 MB/s eta 0:00:00


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig
import torch

## Loading Model

In [3]:
model_name_or_id = "MaralGPT/Maral-7B-alpha-1"

In [5]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!
The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

...

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

## Model Structure

In [6]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      

## Prompt Format

This model, uses _Guanaco_ format, which is like this:

```
### Human: <prompt>
### Assistant: <answer>
```

So in the below cell, you can easily modify the prompt without messing with the format.

In [27]:
prompt = "چطوری یک خانواده میتواند خودش را نابود کند؟"
prompt = f"### Human:{prompt}\n### Assistant:"

In [28]:
inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

### Generation Config

This cell, is a simple and easy way to tweak the configurations for text generation.

In [29]:
generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.8,
    max_new_tokens=350,
    pad_token_id=tokenizer.eos_token_id
)

In [31]:
outputs = model.generate(**inputs, generation_config=generation_config)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### Human:چطوری یک خانواده میتواند خودش را نابود کند؟
### Assistant: یک خانواده می تواند خود را نابود کند تا از اینکه انتخاب ایجاد شده است برای ایجاد یک خانواده با ایجاد این خانواده استفاده کند. این ایجاد شده است برای ایجاد یک خانواده با ایجاد این خانواده استفاده کند. این ایجاد شده است برای ایجاد یک خانواده با ایجاد این خانواده استفاده کند. این ایجاد شده است برای ایجاد یک خانواده با ایجاد این خانواده استفاده کند. ای
